In [1]:
import pandas as pd
import json
import gzip
import numpy as np
import random
from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from surprise import SVD,SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

from surprise.model_selection import GridSearchCV
from tqdm import tqdm
import matplotlib.pyplot as plt
from surprise.model_selection.validation import cross_validate

In [2]:
df = pd.read_csv(r"C:\Users\pruth\Downloads\final project\baseline\datasets\google_reviews.csv")

In [3]:
df

,user_id,name,time,rating,text,pics,resp,gmap_id
0,111435346941066248277,Jenn Thomas,1602893531994,5,"When knowledge is key and kindness matters, Ni...",NaN,NaN,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
1,103613723635264236030,Michelle Barta,1575991509006,5,The entire team is outstanding! They are profe...,NaN,"{'time': 1580484156696, 'text': 'We are lucky ...",0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
2,113556685435236755030,Bethany Baldes,1626981484302,5,NaN,NaN,NaN,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
3,102235352482458236558,nichole vogt,1583381337082,5,NaN,NaN,NaN,0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
4,114021734869233847378,Kristi Friday,1571112557173,4,NaN,NaN,"{'time': 1571246498430, 'text': 'Thanks for th...",0x8758dd1ca83449d9:0xb6156dcfc5e04c9b
...,...,...,...,...,...,...,...,...
427803,107899148903686037545,Carlene Calabaza,1614741323428,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4
427804,117387828996525813955,Gizmo,1561931911835,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4
427805,107887136514890657569,Jan Soucek,1501029366785,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4
427806,104765216668868688471,Kyle Leatherow,1565466860108,3,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4


In [6]:
df['gmap_id'].nunique(),df['user_id'].nunique(),df['rating'].count()



(5567, 18201, 427808)

In [7]:
df['rating'].max(),df['rating'].min(),round(df['rating'].sum()/df['rating'].count(),2)



(5, 1, 4.26)

In [8]:
df['rating'].std(),df['rating'].var()



(1.0553967781398852, 1.1138623593080499)

In [9]:
from pandas._libs.missing import NA
sparsity = (df == 0 | df.isna()).any().astype(int).sum() / df.size
sparsity

0.0

In [3]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[["user_id","gmap_id","rating"]], reader)
raw_ratings = data.raw_ratings

# re arranging the dataset
random.shuffle(raw_ratings)

# splitting the data set into 80:20 training and testing data set
train_test_split_index = int(0.8 * len(raw_ratings))
raw_ratings_train = raw_ratings[:train_test_split_index]
raw_ratings_test = raw_ratings[train_test_split_index:]

# constructing the train set
data.raw_ratings = raw_ratings_train  # assign the raw data back

# constructing the test set
testset = data.construct_testset(raw_ratings_test)

In [4]:
"""
SVDpp parameters
N_factors: The number of factors.
N_epochs: The number of iterations of the SGD procedure.
lr_all – The learning rate for all parameters.
reg_all – The regularization term for all parameters.
"""

# constructing different parameter with different range of values
### note: these values are changed in the whole process of training to arrive at this specific values based on rmse scores.
svdpp_params = {
        "n_factors": [10, 50],
    "n_epochs": [10, 50]
    }

# using grid search cv with above parameters with cross validation of 3 and refit = True
grid_search = GridSearchCV(
    NMF,
    param_grid = svdpp_params,
    measures=["rmse"],
    cv=3,
    refit=True,
    n_jobs=-1,
    joblib_verbose=1
)

# fitting the data
grid_search.fit(data)

# getting best model out of the grid search and best parameters
best_model = grid_search.best_estimator["rmse"]
print(grid_search.best_params)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    6.4s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   11.1s finished


{'rmse': {'n_factors': 50, 'n_epochs': 50}}


In [5]:
# pickle the model in case the run time stop in between
import pickle
with open(r"C:\Users\pruth\Downloads\final project\models\google_nmf.pickle", 'wb') as f:
    pickle.dump(best_model, f)

In [6]:

# calculating rmse on test set
testset_predictions = best_model.test(testset)

accuracy.mae(testset_predictions)

MAE:  0.6129


0.6128503535541653

In [7]:
accuracy.rmse(testset_predictions)


RMSE: 1.0209


1.02093455451102